# Loss functions

> To be written.

In [ ]:
#| default_exp loss_functions

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from typing import Union, Optional

import numpy as np
from ddopnew.utils import Parameter

import torch
import torch.nn as nn
import torch.nn.functional as F

TODO:
 - write descriptions
 - specify typings
 - ensure documentation on types, outputs, variable descriptions

In [ ]:
#| export

def pinball_loss(y_true, y_pred, underage_cost, overage_cost):

    if isinstance(underage_cost, Parameter):
        underage_cost = underage_cost.get_value()
    if isinstance(overage_cost, Parameter):
        overage_cost = overage_cost.get_value()
    
    loss = np.maximum(y_true - y_pred, 0) * underage_cost + np.maximum(y_pred - y_true, 0) * overage_cost

    return loss

In [ ]:
#| export

def quantile_loss(y_true, y_pred, quantile):

    if isinstance(quantile, Parameter):
        quantile = quantile.get_value()
    
    # assert shapes
    assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"
    
    loss = np.maximum((y_true - y_pred) * quantile, (y_pred - y_true) * (1 - quantile))

    return loss

In [ ]:
#| export

class TorchQuantileLoss(nn.modules.loss._Loss):

    __constants__ = ['reduction']
    def __init__(self,reduction: str = 'mean') -> None:
        super().__init__(reduction=reduction)

    def forward(self, input, target, quantile):

        if isinstance(quantile, Parameter):
            quantile = quantile.get_value()
        
        quantile = torch.tensor(quantile, dtype=input.dtype, device=input.device)
            
        if not (target.shape[1] == input.shape[1] == quantile.shape[0]):
            warnings.warn(
                f"Mismatch in dimensions: target dimension 2 size ({target.size(2)}), input dimension 2 size ({input.size(2)}), "
                f"and quantile dimension 1 size ({quantile.size(1)}) must be the same. "
                "This will likely lead to incorrect results due to broadcasting. "
                "Please ensure they have the same size.",
                stacklevel=2,
            )

        return quantile_loss(input, target, quantile, reduction=self.reduction)

def quantile_loss(
    input: torch.Tensor,
    target: torch.Tensor,
    quantile: torch.Tensor,
    reduction: str = 'mean',
) -> torch.Tensor:

    if not (target.size() == input.size()):
        warnings.warn(
            f"Using a target size ({target.size()}) that is different to the input size ({input.size()}). "
            "This will likely lead to incorrect results due to broadcasting. "
            "Please ensure they have the same size.",
            stacklevel=2,
        )

    expanded_input, expanded_target = torch.broadcast_tensors(input, target)
    
    loss = torch.max((expanded_target - expanded_input) * quantile, (expanded_input - expanded_target) * (1 - quantile))

    if reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    else:
        raise ValueError(f"reduction={reduction} is not valid")

    return loss

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()